In [265]:
#May require to pip install netCDF4

Documentation: http://www.odip.org/documents/odip/downloads/20/argo-dm-user-manual.pdf

https://archimer.ifremer.fr/doc/00187/29825/68654.pdf 

NetCDF: https://unidata.github.io/netcdf4-python/netCDF4/index.html

In [266]:
import pandas as pd
import numpy as np
import netCDF4

In [309]:
from ftplib import FTP

with FTP("usgodae.org") as ftp:
    ftp.login()
    ftp.cwd('/pub/outgoing/argo/geo/atlantic_ocean/2009/01')
    print(ftp.nlst()[:2])
    x = ftp.nlst()[0]
    print(type(x))
    print(x.split("_")[0])
    ftp.quit()

['20090101_prof.nc', '20090102_prof.nc']
<class 'str'>
20090101


In [268]:
# commented out for later etl to download straight from ftp, open, and push to db
# url='ftp://usgodae.org/pub/outgoing/argo/geo/atlantic_ocean/2020/08/20200801_prof.nc'
# f = urllib.request.urlopen(url)

# turns out you have to use netcdf4 - this is a multidimensional dataset, so flat array packages like xarray doesn't work

argo = netCDF4.Dataset('argo')
argo.variables


{'DATA_TYPE': <class 'netCDF4._netCDF4.Variable'>
 |S1 DATA_TYPE(STRING16)
     long_name: Data type
     conventions: Argo reference table 1
     _FillValue: b' '
 unlimited dimensions: 
 current shape = (16,)
 filling on,
 'FORMAT_VERSION': <class 'netCDF4._netCDF4.Variable'>
 |S1 FORMAT_VERSION(STRING4)
     long_name: File format version
     _FillValue: b' '
 unlimited dimensions: 
 current shape = (4,)
 filling on,
 'HANDBOOK_VERSION': <class 'netCDF4._netCDF4.Variable'>
 |S1 HANDBOOK_VERSION(STRING4)
     long_name: Data handbook version
     _FillValue: b' '
 unlimited dimensions: 
 current shape = (4,)
 filling on,
 'REFERENCE_DATE_TIME': <class 'netCDF4._netCDF4.Variable'>
 |S1 REFERENCE_DATE_TIME(DATE_TIME)
     long_name: Date of reference for Julian days
     conventions: YYYYMMDDHHMISS
     _FillValue: b' '
 unlimited dimensions: 
 current shape = (14,)
 filling on,
 'DATE_CREATION': <class 'netCDF4._netCDF4.Variable'>
 |S1 DATE_CREATION(DATE_TIME)
     long_name: Date of

In [269]:
for key in list(argo.variables.keys()):
    print(list(argo.variables.keys()).index(key), argo.variables[key][:].dtype, argo.variables[key][:].shape, key)

0 |S1 (16,) DATA_TYPE
1 |S1 (4,) FORMAT_VERSION
2 |S1 (4,) HANDBOOK_VERSION
3 |S1 (14,) REFERENCE_DATE_TIME
4 |S1 (14,) DATE_CREATION
5 |S1 (14,) DATE_UPDATE
6 |S1 (115, 8) PLATFORM_NUMBER
7 |S1 (115, 64) PROJECT_NAME
8 |S1 (115, 64) PI_NAME
9 |S1 (115, 4, 16) STATION_PARAMETERS
10 int32 (115,) CYCLE_NUMBER
11 |S1 (115,) DIRECTION
12 |S1 (115, 2) DATA_CENTRE
13 |S1 (115, 32) DC_REFERENCE
14 |S1 (115, 4) DATA_STATE_INDICATOR
15 |S1 (115,) DATA_MODE
16 |S1 (115, 32) PLATFORM_TYPE
17 |S1 (115, 32) FLOAT_SERIAL_NO
18 |S1 (115, 32) FIRMWARE_VERSION
19 |S1 (115, 4) WMO_INST_TYPE
20 float64 (115,) JULD
21 |S1 (115,) JULD_QC
22 float64 (115,) JULD_LOCATION
23 float64 (115,) LATITUDE
24 float64 (115,) LONGITUDE
25 |S1 (115,) POSITION_QC
26 |S1 (115, 8) POSITIONING_SYSTEM
27 |S1 (115,) PROFILE_CNDC_QC
28 |S1 (115,) PROFILE_PRES_QC
29 |S1 (115,) PROFILE_PSAL_QC
30 |S1 (115,) PROFILE_TEMP_QC
31 |S1 (115, 256) VERTICAL_SAMPLING_SCHEME
32 int32 (115,) CONFIG_MISSION_NUMBER
33 float32 (115, 1007) CND

In [301]:
def utf_decoding(array):
    return [x.decode('UTF-8') for x in array.data]

def masked_arrays_decoding(masked_array):
    lst = []
    for idx, row in enumerate(masked_array):
        number = ''.join(utf_decoding(row)).strip()
        lst.append(number)
    return lst

def param_masked_arrays_decoding(masked_array):
    # TODO: improve this horrendous function. The scientific calibrations have 4 dimensions which complicate a bit the automation. 
    calibration = []
    for arrays in range(len(masked_array)):
        lst = []
        for idx, row in enumerate(masked_array[arrays].data):
            temp = []
            for x in range(len(row)):
                temp.append(''.join([y.decode('UTF-8') for y in row[x]]).strip())
            lst.append(temp)
        calibration.append(lst)
    return calibration

tt = {}
for var in list(argo.variables.keys())[:52]: 
    '''
    ['HISTORY_INSTITUTION', 'HISTORY_STEP', 'HISTORY_SOFTWARE', 'HISTORY_SOFTWARE_RELEASE', 'HISTORY_REFERENCE',
    'HISTORY_DATE', 'HISTORY_ACTION', 'HISTORY_PARAMETER', 'HISTORY_START_PRES', 'HISTORY_STOP_PRES', 
    'HISTORY_PREVIOUS_VALUE', 'HISTORY_QCTEST'] (argo.variables.key()[52:]) are empty so we should not bother with them.
    '''
    if len(argo.variables[var][:]) <= 16:
    # Deals with the first variables which have a single information in them e.g. ['DATA_TYPE', 'FORMAT_VERSION', 'HANDBOOK_VERSION', 'REFERENCE_DATE_TIME', 'DATE_CREATION', 'DATE_UPDATE']
        tt[var] = ''.join(utf_decoding(argo.variables[var][:])).strip()
    elif argo.variables[var][:].dtype != '|S1' and argo.variables[var][:].ndim == 1:
        tt[var] = argo.variables[var][:]
    elif argo.variables[var][:].dtype != '|S1' and argo.variables[var][:].ndim > 1:
        tt[var] = list(argo.variables[var][:].data)
    elif argo.variables[var][:].ndim == 1:
        tt[var] = utf_decoding(argo.variables[var][:])
    elif argo.variables[var][:].ndim == 2:
        tt[var] = masked_arrays_decoding(argo.variables[var][:])
    elif argo.variables[var][:].ndim == 3:
        tt[var] = [masked_arrays_decoding(x) for x in argo.variables[var][:]]
    else:
        tt[var] = param_masked_arrays_decoding(argo.variables[var][:])
    
    

argo_df = pd.DataFrame(tt)
latitude_limits = (argo_df["LATITUDE"] > 38) & (argo_df["LATITUDE"] < 59)
longitude_limits = (argo_df["LONGITUDE"] > -70) & (argo_df["LONGITUDE"] < -35)

#Restrict floats to region surrounding the St Lawrence Estuary. This is a fairly large area but can be further filtered if needed. 
#Keep only columns we will need and rename columns that will be retained after processing. 
argo_df = (argo_df[latitude_limits & longitude_limits]
           [["CYCLE_NUMBER","DATE_CREATION", "JULD_ADJUSTED", "PLATFORM_NUMBER", "LATITUDE", "LONGITUDE", "PRES", "PRES_ADJUSTED", "TEMP", "TEMP_ADJUSTED", 
             "PSAL", "PSAL_ADJUSTED", "PROJECT_NAME"]]
           .rename(columns={"PLATFORM_NUMBER":"float_id", "LATITUDE":"latitude", "LONGITUDE":"longitude", "PROJECT_NAME": "data_source"})
          )
           
argo_df

KeyError: "['JULD_ADJUSTED'] not in index"

In [300]:
def unnesting(df, columns):
    '''Unnest the parameter data for Pressure, Salinity and Temperature. Pressure will be used to calculate depth.'''
    idx = df.index.repeat(df[columns[0]].str.len())
    df1 = pd.concat([pd.DataFrame({x: np.concatenate(df[x].values)}) for x in columns], axis=1)
    df1.index = idx

    return df1.join(df.drop(columns, 1), how='left')

def param(columns):
    '''Removes NaN values (99999.00) and select measured parameter when the adjusted parameters is NaN.'''
    non_adjusted = columns[0]
    adjusted = columns[1]
    param = []
    for x in range(len(columns[0])):
        if adjusted.iloc[x] >= 99999:
            param.append(non_adjusted.iloc[x])
        elif adjusted.iloc[x] >= 99999:
            param.append("NaN")
        else:
            param.append(non_adjusted.iloc[x])
    return param

to_unnest = ['PRES', 'PRES_ADJUSTED','PSAL', 'PSAL_ADJUSTED', 'TEMP', 'TEMP_ADJUSTED']       

argo_df["data_date"] = pd.to_datetime(argo_df["JULD"])
unnested_argo = unnesting(argo_df, to_unnest)
# print(len(unnested_argo[unnested_argo["PRES"] != 99999].dropna()))
# print(len(unnested_argo[unnested_argo["PSAL"] != 99999].dropna()))
# print(len(unnested_argo[unnested_argo["TEMP"] != 99999].dropna()))
unnested_argo["temperature"] = param([unnested_argo["TEMP"], unnested_argo["TEMP_ADJUSTED"]])
unnested_argo["salinity"] = param([unnested_argo["PSAL"], unnested_argo["PSAL_ADJUSTED"]])
# For every 10 meters you go down, the pressure increases by one atmosphere (approx 1 decibar) so with pressure recorded
# in decibar we can assume depth is equivalent.
unnested_argo["depth"] = param([unnested_argo["PRES"], unnested_argo["PRES_ADJUSTED"]])
t = unnested_argo[["data_date", "float_id", "latitude", "longitude", "depth", "temperature", "salinity"]]
t = t[t != 99999].dropna()
t

,data_date,float_id,latitude,longitude,depth,temperature,salinity
26,1970-01-01 00:00:00.000023012,4901218,41.652000,-41.517000,5.000000,17.392000,36.308998
26,1970-01-01 00:00:00.000023012,4901218,41.652000,-41.517000,10.000000,17.393999,36.308998
26,1970-01-01 00:00:00.000023012,4901218,41.652000,-41.517000,15.000000,17.391001,36.306999
26,1970-01-01 00:00:00.000023012,4901218,41.652000,-41.517000,20.000000,17.393999,36.306999
26,1970-01-01 00:00:00.000023012,4901218,41.652000,-41.517000,25.000000,17.395000,36.307999
...,...,...,...,...,...,...,...
114,1970-01-01 00:00:00.000023012,4901168,51.250999,-38.369999,1800.000000,3.523000,34.932999
114,1970-01-01 00:00:00.000023012,4901168,51.250999,-38.369999,1850.400024,3.478000,34.932999
114,1970-01-01 00:00:00.000023012,4901168,51.250999,-38.369999,1899.699951,3.458000,34.932999
114,1970-01-01 00:00:00.000023012,4901168,51.250999,-38.369999,1949.300049,3.422000,34.931000


In [272]:
bins = range(0,int(t["depth"].max()+100),100)
#t['binned'] = pd.cut(t['depth'], bins)
t = t.groupby(["data_date", "float_id", pd.cut(t.depth, bins)]).mean().drop("depth", axis=1).reset_index().dropna()
t

,data_date,float_id,depth,latitude,longitude,temperature,salinity
0,2020-09-03 19:31:27,1901214,"(0, 100]",51.435,-39.811,6.750375,34.574313
1,2020-09-03 19:31:27,1901214,"(100, 200]",51.435,-39.811,5.860000,34.751600
2,2020-09-03 19:31:27,1901214,"(200, 300]",51.435,-39.811,4.986900,34.833900
3,2020-09-03 19:31:27,1901214,"(300, 400]",51.435,-39.811,4.536400,34.863000
4,2020-09-03 19:31:27,1901214,"(400, 500]",51.435,-39.811,4.462400,34.892900
...,...,...,...,...,...,...,...
67,2020-09-03 19:31:27,4901524,"(700, 800]",39.373,-64.506,6.521000,35.040001
68,2020-09-03 19:31:27,4901524,"(800, 900]",39.373,-64.506,5.829000,35.027000
69,2020-09-03 19:31:27,4901524,"(900, 1000]",39.373,-64.506,5.268500,35.025499
70,2020-09-03 19:31:27,4901524,"(1000, 1100]",39.373,-64.506,4.765000,35.006001


In [273]:
t["data_source"] = "Argo Project"
t

,data_date,float_id,depth,latitude,longitude,temperature,salinity,data_source
0,2020-09-03 19:31:27,1901214,"(0, 100]",51.435,-39.811,6.750375,34.574313,Argo Project
1,2020-09-03 19:31:27,1901214,"(100, 200]",51.435,-39.811,5.860000,34.751600,Argo Project
2,2020-09-03 19:31:27,1901214,"(200, 300]",51.435,-39.811,4.986900,34.833900,Argo Project
3,2020-09-03 19:31:27,1901214,"(300, 400]",51.435,-39.811,4.536400,34.863000,Argo Project
4,2020-09-03 19:31:27,1901214,"(400, 500]",51.435,-39.811,4.462400,34.892900,Argo Project
...,...,...,...,...,...,...,...,...
67,2020-09-03 19:31:27,4901524,"(700, 800]",39.373,-64.506,6.521000,35.040001,Argo Project
68,2020-09-03 19:31:27,4901524,"(800, 900]",39.373,-64.506,5.829000,35.027000,Argo Project
69,2020-09-03 19:31:27,4901524,"(900, 1000]",39.373,-64.506,5.268500,35.025499,Argo Project
70,2020-09-03 19:31:27,4901524,"(1000, 1100]",39.373,-64.506,4.765000,35.006001,Argo Project


In [297]:
t["depth"] = t["depth"].astype(str)
t["data_date"] = pd.to_datetime(t["data_date"])
t#.dtypes

,data_date,float_id,latitude,longitude,depth,temperature,salinity
26,2020-09-03 19:31:27,4901218,41.652000,-41.517000,5.0,17.392000,36.308998
26,2020-09-03 19:31:27,4901218,41.652000,-41.517000,10.0,17.393999,36.308998
26,2020-09-03 19:31:27,4901218,41.652000,-41.517000,15.0,17.391001,36.306999
26,2020-09-03 19:31:27,4901218,41.652000,-41.517000,20.0,17.393999,36.306999
26,2020-09-03 19:31:27,4901218,41.652000,-41.517000,25.0,17.395000,36.307999
...,...,...,...,...,...,...,...
114,2020-09-03 19:31:27,4901168,51.250999,-38.369999,1800.0,3.523000,34.932999
114,2020-09-03 19:31:27,4901168,51.250999,-38.369999,1850.4000244140625,3.478000,34.932999
114,2020-09-03 19:31:27,4901168,51.250999,-38.369999,1899.699951171875,3.458000,34.932999
114,2020-09-03 19:31:27,4901168,51.250999,-38.369999,1949.300048828125,3.422000,34.931000


In [293]:
t["data_date"] = pd.to_datetime(t["data_date"])
t

,data_date,float_id,latitude,longitude,depth,temperature,salinity
26,2020-09-03,4901218,41.652000,-41.517000,5.0,17.392000,36.308998
26,2020-09-03,4901218,41.652000,-41.517000,10.0,17.393999,36.308998
26,2020-09-03,4901218,41.652000,-41.517000,15.0,17.391001,36.306999
26,2020-09-03,4901218,41.652000,-41.517000,20.0,17.393999,36.306999
26,2020-09-03,4901218,41.652000,-41.517000,25.0,17.395000,36.307999
...,...,...,...,...,...,...,...
114,2020-09-03,4901168,51.250999,-38.369999,1800.0,3.523000,34.932999
114,2020-09-03,4901168,51.250999,-38.369999,1850.4000244140625,3.478000,34.932999
114,2020-09-03,4901168,51.250999,-38.369999,1899.699951171875,3.458000,34.932999
114,2020-09-03,4901168,51.250999,-38.369999,1949.300048828125,3.422000,34.931000


In [294]:
t.dtypes

data_date      datetime64[ns]
float_id               object
latitude              float64
longitude             float64
depth                  object
temperature           float64
salinity              float64
dtype: object

In [10]:
# TODO: decide whether expanding all parameters data before or after writing to DB. Param data contains a lot of noise (e.g. 99999.0000 values) 
# Is depth indicated somewhere? or all measures taken at specific depth? TBC from documentation

In [212]:
import folium
m = folium.Map(location=[50, -60], zoom_start=4.5, tiles='Stamen Terrain')

for flt in t.index:
    folium.Marker(location=[t["latitude"].iloc[flt], t["longitude"].iloc[flt]]).add_to(m)

m

IndexError: single positional indexer is out-of-bounds

In [12]:
temp_df = argo_df[['TEMP']].copy()
temp_df

,TEMP
0,"[26.675, 26.67, 26.663, 26.662, 26.671, 26.669..."
1,"[27.102, 27.103, 27.103, 27.104, 27.104, 27.10..."
2,"[23.352, 23.352, 23.349, 23.348, 23.355, 23.35..."
3,"[23.377, 23.377, 23.382, 23.392, 23.395, 23.39..."
4,"[19.215, 19.229, 19.234, 19.233, 19.23, 19.226..."
...,...
137,"[4.478, 4.476, 4.475, 4.476, 4.473, 4.475, 4.4..."
138,"[4.35, 4.349, 4.35, 4.352, 4.352, 4.352, 4.352..."
139,"[3.674, 3.68, 3.689, 3.691, 3.691, 3.692, 3.69..."
140,"[14.644, 14.665, 14.678, 14.675, 14.678, 14.68..."


In [13]:
df_temp_test = pd.DataFrame(temp_df['TEMP'].to_list())
df_temp_test = df_temp_test.T.replace(99999.0000, None)
# A lot of temperature lists include 99999.0000 values which we need to get rid of
for col in df_temp_test.columns:
    print(f'{col}: min {df_temp_test[col].min()}, max {df_temp_test[col].max()}, mean {df_temp_test[col].mean()}')

0: min 5.315000057220459, max 26.690000534057617, mean 6.271593895368071
1: min 3.5369999408721924, max 27.106000900268555, mean 4.702252833413354
2: min 3.6659998893737793, max 23.368000030517578, mean 5.499650800285495
3: min 3.703000068664551, max 23.399999618530273, mean 5.5425627958946695
4: min 3.7829999923706055, max 19.236000061035156, mean 5.517728609493246
5: min 3.86899995803833, max 25.993000030517578, mean 5.412251720689747
6: min 2.447999954223633, max 9.5649995803833, mean 2.8920603186852047
7: min 3.746999979019165, max 18.350000381469727, mean 4.848009606106817
8: min 3.571000099182129, max 28.097999572753906, mean 4.861038892545421
9: min 0.9089999794960022, max 27.136999130249023, mean 3.3207352862716473
10: min 14.824000358581543, max 23.94300079345703, mean 14.946489790641392
11: min 3.8889999389648438, max 23.940000534057617, mean 5.006264294310439
12: min 5.249000072479248, max 24.63599967956543, mean 6.232784584600455
13: min 1.7790000438690186, max 25.597999572

In [14]:
temp_df = argo_df[['LATITUDE', 'LONGITUDE']]
temp_df['MEAN_TPT'] = pd.Series([df_temp_test[col].mean() for col in df_temp_test.columns])
temp_df

C:\Users\ssung\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,LATITUDE,LONGITUDE,MEAN_TPT
0,9.201430,-37.727650,6.271594
1,-6.972160,-9.659960,4.702253
2,19.470060,-30.365670,5.499651
3,19.456280,-30.361650,5.542563
4,22.753110,-18.078970,5.517729
...,...,...,...
137,54.006447,-50.540653,3.459037
138,58.633392,-39.617931,3.436786
139,57.910954,-54.497654,3.471710
140,41.499115,-49.142590,3.663514


In [15]:
from folium.plugins import HeatMap
HeatMap(data=temp_df[['LATITUDE', 'LONGITUDE', 'MEAN_TPT']].groupby(['LATITUDE', 'LONGITUDE']).sum().reset_index().values.tolist(), radius=25, max_zoom=10).add_to(m)
m

In [ ]:
# TODO: continue looking at https://shapely.readthedocs.io/en/latest/manual.html to find ways to divide multipolygon files into sub-areas. Seem to not be trivial outside of ArcGIS/QGIS.